# Setting up the environment


In [1]:
# import helpful packages for numerical analysis
import sys
import os
import numpy as np

import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_rows = 30

import pprint

# and for plotting
# from plotly import graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.express as px

# and import pelicun classes and methods
# from notpelicun_3_6.base import set_options, convert_to_MultiIndex
# from notpelicun_3_6.assessment import Assessment

from pelicun_3_1.base import convert_to_MultiIndex
from pelicun_3_1.assessment import Assessment

In [2]:
baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/SurrogateModeling/DGP/woodSDA'

## Data Pre-processing

In [3]:
# ID = 's4_96x48_High_Stucco_Plaster_Normal'
ID = 's1_40x30_High_Stucco_GWB_Heavy'
# ID = 's1_40x30_Moderate_HWS_GWB_Heavy'
# pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'PFA.csv']), header=None)
# sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'SDR.csv']), header=None)
# rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'RDR.csv']), header=None)

pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'PFA.csv']), header=None)
sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'SDR.csv']), header=None)
rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'RDR.csv']), header=None)

In [4]:
# ID = 's4_96x48_High_Stucco_GWB'
# pfa = pd.read_csv(os.path.join(baseDir, *['Results', ID, 'edp_outputs', 'PFA.csv']), header=None)
# sdr = pd.read_csv(os.path.join(baseDir, *['Results', ID, 'edp_outputs', 'SDR.csv']), header=None)
# rdr = pd.read_csv(os.path.join(baseDir, *['Results', ID, 'edp_outputs', 'RDR.csv']), header=None)


# col_names_drift = ['IM', 'Direction', 'GM_ID', 'Story_1', 'Story_2', 'Story_3', 'Story_4']
# col_names_pfa = ['IM', 'Direction', 'GM_ID', 'Story_0', 'Story_1', 'Story_2', 'Story_3', 'Story_4']
# sdr = sdr.rename(columns= dict(zip(sdr.columns, col_names_drift)))
# pfa = pfa.rename(columns = dict(zip(pfa.columns, col_names_pfa)))
# rdr = sdr.rename(columns= dict(zip(rdr.columns, col_names_drift)))
sdr.head()

,0,1,2,3
0,1,1,1,0.000181
1,1,1,2,0.000338
2,1,1,3,0.000405
3,1,1,4,0.000508
4,1,1,5,0.000708


In [5]:
sdr.shape

(100, 4)

In [6]:
pfa.head()

,0,1,2,3,4
0,1,1,1,0.033424,0.040579
1,1,1,2,0.057795,0.091112
2,1,1,3,0.045554,0.073283
3,1,1,4,0.083265,0.123174
4,1,1,5,0.097729,0.190047


In [7]:
pfa.shape

(100, 5)

In [8]:
index_name = []
median = []
log_std = []
#1-PFA-0-1
for stripe in pfa[0].unique():
    churn_df = pfa[pfa[0]==stripe]
    story_name = 0 
    for story in churn_df.columns:
        for uniq_dir in pfa[1].unique():
            if story >=3:
                story_name = story - 3
                index_name.append('%s-PFA-%s-%s'%(stripe, story_name, uniq_dir))
                median.append(pfa[(pfa[0]==stripe) & (pfa[1]==uniq_dir)][story].median())
                log_std.append(pfa[(pfa[0]==stripe) & (pfa[1]==uniq_dir)][story].std())
d = {
    'idx': index_name, 
    'median': median, 
    'log_std': log_std
}
df_pfa = pd.DataFrame(d)
df_pfa = df_pfa.set_index('idx')
df_pfa.head()

,median,log_std
idx,,
1-PFA-0-1,0.066539,0.026599
1-PFA-0-2,0.067214,0.033207
1-PFA-1-1,0.095921,0.053826
1-PFA-1-2,0.110298,0.049966


In [9]:
df_pfa.shape

(4, 2)

In [10]:
index_name = []
median = []
log_std = []
#1-PFA-0-1
for stripe in sdr[0].unique():
    churn_df = sdr[sdr[0]==stripe]
    story_name = 0 
    for story in churn_df.columns:
        for uniq_dir in sdr[1].unique():
            if story >=3:
                story_name = story - 2
                index_name.append('%s-PID-%s-%s'%(stripe, story_name, uniq_dir))
                median.append(sdr[(sdr[0]==stripe) & (sdr[1]==uniq_dir)][story].median())
                log_std.append(sdr[(sdr[0]==stripe) & (sdr[1]==uniq_dir)][story].std())
d = {
    'idx': index_name, 
    'median': median, 
    'log_std': log_std
}
df_sdr = pd.DataFrame(d)
df_sdr = df_sdr.set_index('idx')
df_sdr.head()

,median,log_std
idx,,
1-PID-1-1,0.000408,0.000201
1-PID-1-2,0.000602,0.000246


In [11]:
baseline_num = 3 if 'PFA'=='PFA' else 2
baseline_num

3

In [12]:
df_sdr.shape

(2, 2)

In [13]:
df_pfa.shape

(4, 2)

In [14]:
df_comb = pd.concat([df_pfa, df_sdr])
df_comb.head()

,median,log_std
idx,,
1-PFA-0-1,0.066539,0.026599
1-PFA-0-2,0.067214,0.033207
1-PFA-1-1,0.095921,0.053826
1-PFA-1-2,0.110298,0.049966
1-PID-1-1,0.000408,0.000201


In [15]:
df_comb.shape

(6, 2)

In [16]:
raw_demands = convert_to_MultiIndex(df_comb, axis=0)
raw_demands.index.names = ['stripe','type','loc','dir']
raw_demands.head(n=30)

median   log_std
stripe type loc dir                    
1      PFA  0   1    0.066539  0.026599
                2    0.067214  0.033207
            1   1    0.095921  0.053826
                2    0.110298  0.049966
       PID  1   1    0.000408  0.000201
                2    0.000602  0.000246

In [17]:
os.getcwd()

'/Users/laxmandahal/Desktop/UCLA/Phd/Research/SurrogateModeling/DGP/woodSDA/Codes/Loss_Pelicun'

In [18]:
# raw_demands.to_csv('raw_demand_debug.csv')

In [19]:
stripe = '1'
stripe_demands = raw_demands.loc[stripe,:]
stripe_demands

median   log_std
type loc dir                    
PFA  0   1    0.066539  0.026599
         2    0.067214  0.033207
     1   1    0.095921  0.053826
         2    0.110298  0.049966
PID  1   1    0.000408  0.000201
         2    0.000602  0.000246

In [20]:
# we'll use stripe 3 for this example
stripe = '1'
stripe_demands = raw_demands.loc[stripe,:]

# units - - - - - - - - - - - - - - - - - - - - - - - -  
stripe_demands.insert(0, 'Units',"")

# PFA is in "g" in this example, while PID is "rad"
stripe_demands.loc['PFA','Units'] = 'g'
stripe_demands.loc['PID','Units'] = 'rad'

# distribution family  - - - - - - - - - - - - - - - - -  
stripe_demands.insert(1, 'Family',"")

# we assume lognormal distribution for all demand marginals
stripe_demands['Family'] = 'lognormal'

# distribution parameters  - - - - - - - - - - - - - - -
# pelicun uses generic parameter names to handle various distributions within the same data structure
# we need to rename the parameter columns as follows:
# median -> theta_0
# log_std -> theta_1
stripe_demands.rename(columns = {'median': 'Theta_0'}, inplace=True)
stripe_demands.rename(columns = {'log_std': 'Theta_1'}, inplace=True)

stripe_demands

Units     Family   Theta_0   Theta_1
type loc dir                                     
PFA  0   1       g  lognormal  0.066539  0.026599
         2       g  lognormal  0.067214  0.033207
     1   1       g  lognormal  0.095921  0.053826
         2       g  lognormal  0.110298  0.049966
PID  1   1     rad  lognormal  0.000408  0.000201
         2     rad  lognormal  0.000602  0.000246

In [21]:
# initialize a pelicun Assessment
PAL = Assessment({
    'Verbose':True,
    "PrintLog": True, 
    "Seed": 415})

pelicun 3.1 | 

System Information:
         local time zone: +0545
         start time: 2023-03-20T19:53:40
         python: 3.8.2 (default, Mar 25 2020, 11:22:43) 
         [Clang 4.0.1 (tags/RELEASE_401/final)]
         numpy: 1.21.4
         pandas: 1.4.4
         
         --------------------------------------------------------------------------------
19:53:40 Assessement Started


In [22]:
# prepare a correlation matrix that represents perfect correlation
ndims = stripe_demands.shape[0]
demand_types = stripe_demands.index 

perfect_CORR = pd.DataFrame(
    np.ones((ndims, ndims)),
    columns = demand_types,
    index = demand_types)

# load the demand model
PAL.demand.load_model({'marginals': stripe_demands,
                       'correlation': perfect_CORR})


         --------------------------------------------------------------------------------
19:53:40 Loading demand model...
         Demand model successfully loaded.


In [23]:
# perfect_CORR

In [24]:
# choose a sample size for the analysis
sample_size = 1000

# generate demand sample
PAL.demand.generate_sample({"SampleSize": sample_size})

# extract the generated sample
# Note that calling the save_sample() method is better than directly pulling the 
# sample attribute from the demand object because the save_sample method converts
# demand units back to the ones you specified when loading in the demands.
# demand_sample = PAL.demand.save_sample()

# demand_sample.head()

         --------------------------------------------------------------------------------
19:53:40 Generating sample from demand variables...
         
         6 random variables created.
         
         Correlations between 6 random variables successfully defined.
         
         Successfully generated 1000 realizations.


In [25]:

# extract the generated sample
# Note that calling the save_sample() method is better than directly pulling the 
# sample attribute from the demand object because the save_sample method converts
# demand units back to the ones you specified when loading in the demands.
demand_sample = PAL.demand.save_sample()

demand_sample.head()

         --------------------------------------------------------------------------------


type       PFA                                     PID          
loc          0                   1                   1          
dir          1         2         1         2         1         2
0     0.067488  0.068413  0.098710  0.113272  0.000408  0.000602
1     0.064469  0.064614  0.089978  0.103940  0.000408  0.000602
2     0.066929  0.067706  0.097061  0.111515  0.000408  0.000602
3     0.066363  0.066992  0.095407  0.109749  0.000408  0.000602
4     0.063570  0.063490  0.087456  0.101233  0.000408  0.000602

In [26]:
demand_sample.shape

(1000, 6)

In [27]:

# datDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/woodSDA/Resources/PELICUN_example/Example01_FEMA_P58_Introduction'

# PAL.demand.save_sample(os.path.join(datDir, 'edp_demand_sample.csv'))

In [28]:
# get residual drift estimates 
delta_y = 0.0075
PID = demand_sample['PID']

RID = PAL.demand.estimate_RID(PID, {'yield_drift': delta_y})

# and join them with the demand_sample
demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

# add spectral acceleration
Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[int(stripe)-1]

demand_sample_ext.describe().T

count      mean           std       min       25%       50%       75%       max
type    loc dir                                                                                  
PFA     0   1    1000.0  0.066563  1.769690e-03  0.060803  0.065359  0.066540  0.067744  0.072520
            2    1000.0  0.067251  2.232337e-03  0.060059  0.065729  0.067216  0.068737  0.074839
        1   1    1000.0  0.096059  5.170606e-03  0.079925  0.092508  0.095924  0.099467  0.114170
            2    1000.0  0.110435  5.517534e-03  0.093114  0.106650  0.110301  0.114078  0.129653
PID     1   1    1000.0  0.000408  8.203530e-08  0.000408  0.000408  0.000408  0.000408  0.000409
            2    1000.0  0.000602  1.479827e-07  0.000601  0.000602  0.000602  0.000602  0.000602
RID     1   1    1000.0  0.000000  0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000
            2    1000.0  0.000000  0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000
SA_1.13 0   1    1000.0  0.158000  1.582859e-15  0.158000  0.158000  0.158000  0.158000  0.158000

In [29]:
# demand_sample_ext.to_csv('epd_demand_sample_with_residual.csv')

In [30]:
# demand_sample_ext.head()

In [31]:
# add units to the data 
demand_sample_ext.T.insert(0, 'Units',"")

# PFA and SA are in "g" in this example, while PID and RID are "rad"
demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

display(demand_sample_ext)

PAL.demand.load_sample(demand_sample_ext)

type        PFA                                     PID            RID      SA_1.13
loc           0                   1                   1              1            0
dir           1         2         1         2         1         2    1    2       1
0      0.067488  0.068413   0.09871  0.113272  0.000408  0.000602  0.0  0.0   0.158
1      0.064469  0.064614  0.089978   0.10394  0.000408  0.000602  0.0  0.0   0.158
2      0.066929  0.067706  0.097061  0.111515  0.000408  0.000602  0.0  0.0   0.158
3      0.066363  0.066992  0.095407  0.109749  0.000408  0.000602  0.0  0.0   0.158
4       0.06357   0.06349  0.087456  0.101233  0.000408  0.000602  0.0  0.0   0.158
...         ...       ...       ...       ...       ...       ...  ...  ...     ...
996    0.063022  0.062808  0.085938  0.099601  0.000408  0.000602  0.0  0.0   0.158
997    0.065818  0.066306  0.093828  0.108062  0.000408  0.000602  0.0  0.0   0.158
998    0.068809  0.070088  0.102657   0.11747  0.000408  0.000602  0.0  0.0   0.158
999    0.066461  0.067115  0.095691  0.110053  0.000408  0.000602  0.0  0.0   0.158
Units         g         g         g         g       rad       rad  rad  rad       g

[1001 rows x 9 columns]

         --------------------------------------------------------------------------------
19:53:41 Loading demand data...
WARNING in pandas/core/indexing.py at line 2120
In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)

         Removing whitespace from header...
         Demand data successfully parsed.
         Removing whitespace from header...
         Demand units successfully parsed.


In [32]:
# PAL.demand.save_sample(os.path.join(datDir, 'epd_demand_sample_with_residual.csv'))

## 3.3 Damage

### Define asset model

The asset model assigns components to the building and defines where they are and how much of each component is at each location. 

In [33]:
# load the component configuration
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
# baselineID = 's4_96x48'
baselineID = 's1_40x30'
cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                    baselineID,
                                                    'ComponentsList',
                                                    'components_list_marginals.csv']), index_col=1)

display(cmp_marginals.head(15))
print("...")
cmp_marginals.tail(10)

,cmpID,Description,Units,Theta_0,Blocks,Family,Theta_1,Location,Direction,Incomplete,Changelog
ID,,,,,,,,,,,
B.10.71.002,B1071.002,Light framed wood walls with structural panel ...,SF,100,NaN,NaN,NaN,all,"1,2",False,NaN
B.10.71.041,B1071.041,"Exterior Wall - Type: Gypsum with wood studs, ...",LF,100,NaN,NaN,NaN,all,"1,2",False,NaN
B.20.11.111,B2011.111,Exterior Wall - Light framed wood walls with s...,SF,100,NaN,NaN,NaN,all,"1,2",False,NaN
C.10.11.011a,C1011.011a,"Wall Partition, Type: Gypsum with wood studs, ...",LF,100,NaN,lognormal,0.4,all,"1,2",False,NaN
C.10.11.001a,C1011.001a,"Wall Partition, Type: Gypsum with metal studs,...",LF,100,NaN,lognormal,0.4,all,"1,2",False,NaN
D.20.21.013a,D2021.013a,Cold or Hot Potable - Small Diameter Threaded ...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN
D.20.21.013b,D2021.013b,Cold or Hot Potable - Small Diameter Threaded ...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN
D.20.21.023a,D2021.023a,Cold or Hot Potable Water Piping (dia > 2.5 in...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN
D.20.21.023b,D2021.023b,Cold or Hot Potable Water Piping (dia > 2.5 in...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN


...


,cmpID,Description,Units,Theta_0,Blocks,Family,Theta_1,Location,Direction,Incomplete,Changelog
ID,,,,,,,,,,,
D.20.21.023b,D2021.023b,Cold or Hot Potable Water Piping (dia > 2.5 in...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN
D.20.31.013b,D2031.013b,Sanitary Waste Piping - Cast Iron w/flexible c...,LF,1000,NaN,lognormal,0.4,all,"1,2",True,"changed beta from undefined to 0.4, changed ""i..."
D.30.31.013a,D3031.013a,Chiller - Capacity: < 100 Ton - Equipment that...,EA,1,NaN,NaN,NaN,all,1,True,"added 0.8g as MedianEDP, 0.25 as beta, changed..."
D.30.41.011c,D3041.011c,HVAC Galvanized Sheet Metal Ducting less than ...,LF,1000,NaN,lognormal,0.4,all,"1,2",False,NaN
D.30.41.032c,D3041.032c,HVAC Drops / Diffusers without ceilings - supp...,EA,10,NaN,NaN,NaN,roof,1,False,NaN
D.30.41.102b,D3041.102b,HVAC Fan - Capacity: all - Vibration isolated ...,EA,1,NaN,NaN,NaN,roof,1,False,NaN
D.30.41.103a,D3041.103a,HVAC Fan - Capacity: all - Equipment that is e...,EA,1,NaN,NaN,NaN,roof,1,True,NaN
excessiveRID,NaN,excessive residual drift,EA,1,NaN,NaN,NaN,all,"1,2",NaN,NaN
collapse,NaN,collapsed building,EA,1,NaN,NaN,NaN,0,1,NaN,NaN


In [34]:
# to make the convenience keywords work in the model, 
# we need to specify the number of stories
PAL.stories = 1

# now load the model into Pelicun
PAL.asset.load_cmp_model({'marginals': cmp_marginals})

         --------------------------------------------------------------------------------
19:53:41 Loading component model...
         
         Parsing model file to characterize each component block
         Model parameters successfully parsed. 30 performance groups identified
         Converting model parameters to internal units...
         Model parameters successfully loaded.
         
         Component model marginal distributions:
                              Units Family    Theta_0  Theta_1  Blocks
         cmp          loc dir                                         
         B.10.71.002  1   1      SF    NaN   9.290304      NaN     1.0
                          2      SF    NaN   9.290304      NaN     1.0
         B.10.71.041  1   1      LF    NaN  30.480000      NaN     1.0
                          2      LF    NaN  30.480000      NaN     1.0
         B.20.11.111  1   1      SF    NaN   9.290304      NaN     1.0
         ...                    ...    ...        ...     

In [35]:
cmp_marginals.shape

(18, 11)

In [36]:
# let's take a look at the generated marginal parameters
# PAL.asset.cmp_marginal_params.loc['B.10.41.002a',:]
# PAL.asset.cmp_marginal_params.loc['B.10.71.001',:]

### 3.3.2 Sample asset distribution

In this example, the quantities are identical for every realization. We still need to generate a component quantity sample because the calculations in Pelicun expect an array of component quantity realizations. The sample size for the component quantities is automatically inferred from the demand sample. If such a sample is not available, you need to provide a sample size as the first argument of the generate_cmp_sample method.

The table below shows the statistics for each Performance Group's quantities. Notice the zero standard deviation and that the minimum and maximum values are identical - this confirms that the quantities are deterministic.

We could edit this sample and load the edited version back to Pelicun like we did for the Demands earlier.

In [37]:
# Generate the component quantity sample
PAL.asset.generate_cmp_sample()

# get the component quantity sample - again, use the save function to convert units
cmp_sample = PAL.asset.save_cmp_sample()

cmp_sample.describe()

         --------------------------------------------------------------------------------
19:53:41 Generating sample from component quantity variables...
         
         30 random variables created.
         
         Successfully generated 1000 realizations.
         --------------------------------------------------------------------------------


cmp   B.10.71.002         B.10.71.041         B.20.11.111         C.10.11.001a              C.10.11.011a              D.20.21.013a              D.20.21.013b              D.20.21.023a              D.20.21.023b              D.20.31.013b              D.30.31.013a D.30.41.011c               \
loc             1                   1                   1                    1                         1                         1                         1                         1                         1                         1                         1            1                
dir             1       2           1       2           1       2            1            2            1            2            1            2            1            2            1            2            1            2            1            2            1            1            2   
count      1000.0  1000.0      1000.0  1000.0      1000.0  1000.0  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000       1000.0  1000.000000  1000.000000   
mean        100.0   100.0       100.0   100.0       100.0   100.0   108.285905   108.310225   108.277068   108.280973  1082.977888  1082.909925  1083.638892  1083.085665  1083.059397  1082.918828  1082.822908  1082.816426  1085.287267  1083.113286          1.0  1083.817885  1082.871562   
std           0.0     0.0         0.0     0.0         0.0     0.0    44.882067    45.043524    44.904372    44.847419   449.458863   449.087977   453.484430   450.762914   450.357412   449.240452   448.969110   448.689880   466.864900   449.833679          0.0   454.356962   448.897048   
min         100.0   100.0       100.0   100.0       100.0   100.0    25.520569    25.567868    20.469358    27.178054   266.242223   254.236508   280.837971   230.277897   211.364364   254.506345   232.008273   214.376281   275.279242   286.666421          1.0   280.400091   284.197147   
25%         100.0   100.0       100.0   100.0       100.0   100.0    76.361962    76.350401    76.390689    76.402128   763.709423   763.996537   763.495931   764.105571   763.755415   763.844686   763.378574   763.993383   763.969974   763.533376          1.0   763.832483   763.600840   
50%         100.0   100.0       100.0   100.0       100.0   100.0   100.017016    99.985903   100.015843   100.031448   999.867020   999.696397  1000.166414  1000.014049   999.976259   999.900425   999.758069   999.927917  1000.289855  1000.039491          1.0  1000.066543  1000.171842   
75%         100.0   100.0       100.0   100.0       100.0   100.0   130.902210   130.936571   130.993592   130.922022  1309.224811  1309.422106  1309.183618  1309.517573  1309.210125  1310.068726  1309.284216  1309.736608  1309.764660  1309.000486          1.0  1309.616449  1308.823700   
max         100.0   100.0       100.0   100.0       100.0   100.0   359.811993   362.833838   348.114101   346.981164  3521.740066  3502.106968  4226.809188  3896.574554  3828.506603  3610.050597  3485.284024  3517.649896  5751.665205  3496.658338          1.0  4322.955276  3499.562928   

cmp   D.30.41.032c D.30.41.102b D.30.41.103a collapse excessiveRID         irreparable  
loc              2            2            2        0            1                   0  
dir              1            1            1        1            1       2           1  
count       1000.0       1000.0       1000.0   1000.0       1000.0  1000.0      1000.0  
mean          10.0          1.0          1.0      1.0          1.0     1.0         1.0  
std            0.0          0.0          0.0      0.0          0.0     0.0         0.0  
min           10.0          1.0          1.0      1.0          1.0     1.0         1.0  
25%           10.0          1.0          1.0      1.0          1.0     1.0         1.0  
50%           10.0          1.0          1.0      1.0          1.0     1.0         1.0  
75%     

In [38]:
cmp_sample.shape

(1000, 30)

In [39]:
# PAL.asset.save_cmp_sample(os.path.join(datDir, 'component_sample.csv'))

### 3.3.3 Define component fragilities

In [40]:
# review the damage model - in this example: fragility functions
P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')

display(P58_data.head(3))

print(P58_data['Incomplete'].sum(),' incomplete component fragility definitions')

Demand                                          Incomplete                  LS1                                           LS2                                           LS3                                           LS4                       
             Directional Offset                         Type Unit              DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1
ID                                                                                                                                                                                                                                                                
B.10.31.001          1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.08     0.4                NaN  lognormal    0.11     0.4                NaN    NaN     NaN     NaN
B.10.31.011a         1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.07     0.4                NaN  lognormal    0.10     0.4                NaN    NaN     NaN     NaN
B.10.31.011b         1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.07     0.4                NaN  lognormal    0.10     0.4                NaN    NaN     NaN     NaN

185.0  incomplete component fragility definitions


In [41]:
# note that we drop the last three components here (excessiveRID, irreparable, and collapse) 
# because they are not part of P58
cmp_list = cmp_marginals.index.unique().values[:-3]

P58_data_for_this_assessment = P58_data.loc[cmp_list,:].sort_values('Incomplete', ascending=False)

additional_fragility_db = P58_data_for_this_assessment.loc[
    P58_data_for_this_assessment['Incomplete'] == 1].sort_index() 

additional_fragility_db

Empty DataFrame
Columns: [(Demand, Directional), (Demand, Offset), (Demand, Type), (Demand, Unit), (Incomplete, ), (LS1, DamageStateWeights), (LS1, Family), (LS1, Theta_0), (LS1, Theta_1), (LS2, DamageStateWeights), (LS2, Family), (LS2, Theta_0), (LS2, Theta_1), (LS3, DamageStateWeights), (LS3, Family), (LS3, Theta_0), (LS3, Theta_1), (LS4, DamageStateWeights), (LS4, Family), (LS4, Theta_0), (LS4, Theta_1)]
Index: []

In [42]:
additional_fragility_db.index

Index([], dtype='object', name='ID')

In [43]:
# P58_metadata = PAL.get_default_metadata('fragility_DB_FEMA_P58_2nd')

# pprint.pprint(P58_metadata['D.20.22.013b'])

In [44]:
# additional_fragility_db.loc[['D.20.31.013b', 'D.30.31.013a', 'D.30.41.103a',
#                              'D.30.52.013a', 'D.30.67.012a', 'D.50.12.013d',
#                              'D.50.12.023a'], 
#                            [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.4
# additional_fragility_db.loc[['D.20.31.013b', 'D.30.31.013a', 'D.30.41.103a',
#                              'D.30.52.013a', 'D.30.67.012a', 'D.50.12.013d',
#                              'D.50.12.023a'], 
#                            [('LS1','Theta_0'),('LS2','Theta_0')]] = 1.5 #g

In [45]:
# # We can set the incomplete flag to 0 for these components
# additional_fragility_db['Incomplete'] = 0
# additional_fragility_db

In [46]:
# # D2022.013a, 023a, 023b - Heating, hot water piping and bracing
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc[['D.20.22.013a','D.20.22.023a','D.20.22.023b'],
#                             [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.5

# # D2031.013b - Sanitary Waste piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.31.013b',('LS1','Theta_1')] = 0.5

# # D2061.013b - Steam piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.61.013b',('LS1','Theta_1')] = 0.5

# # D3031.013i - Chiller
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_1')] = 0.5

# # D3031.023i - Cooling Tower
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_1')] = 0.5

# # D3052.013i - Air Handling Unit
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_1')] = 0.5

# # We can set the incomplete flag to 0 for these components
# additional_fragility_db['Incomplete'] = 0

# additional_fragility_db

In [47]:

# irreparable damage
# this is based on the default values in P58
additional_fragility_db.loc[
    'excessiveRID', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

additional_fragility_db.loc[
    'excessiveRID', [('LS1','Family'),
                    ('LS1','Theta_0'),
                    ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

additional_fragility_db.loc[
    'irreparable', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   


# a very high capacity is assigned to avoid damage from demands
additional_fragility_db.loc[
    'irreparable', ('LS1','Theta_0')] = 1e10 

# collapse
# capacity is assigned based on the example in the FEMA P58 background documentation
additional_fragility_db.loc[
    'collapse', [('Demand','Directional'),
                 ('Demand','Offset'),
                 ('Demand','Type'), 
                 ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   


additional_fragility_db.loc[
    'collapse', [('LS1','Family'),
                 ('LS1','Theta_0'),
                 ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

# We set the incomplete flag to 0 for the additional components
additional_fragility_db['Incomplete'] = 0

additional_fragility_db.tail(3)

Demand                                              Incomplete                LS1                                                 LS2                                       LS3                                       LS4                       
             Directional Offset                             Type Unit            DamageStateWeights     Family       Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1
ID                                                                                                                                                                                                                                                                
excessiveRID         1.0    0.0  Residual Interstory Drift Ratio  rad          0                NaN  lognormal  1.000000e-02     0.3                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN
irreparable          1.0    0.0  Peak Spectral Acceleration|1.13    g          0                NaN        NaN  1.000000e+10     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN
collapse             1.0    0.0  Peak Spectral Acceleration|1.13    g          0                NaN  lognormal  1.350000e+00     0.5                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN

### 3.3.4 Load component fragility data

In [48]:
PAL.damage.load_damage_model([
    additional_fragility_db,  # This is the extra fragility data we've just created
    'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
])

         --------------------------------------------------------------------------------
19:53:41 Loading damage model...
         Damage model parameters successfully parsed.


In [49]:
# PAL.damage.load_damage_model([
# #     additional_fragility_db,  # This is the extra fragility data we've just created
#     'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
# ])

## 3.3.5 Damage Process

In [50]:
# FEMA P58 uses the following process:
dmg_process = {
    "1_collapse": {
        "DS1": "ALL_NA"
    },
    "2_excessiveRID": {
        "DS1": "irreparable_DS1"
    }
}

In [51]:
# Now we can run the calculation
PAL.damage.calculate(dmg_process=dmg_process)#, block_batch_size=100) #- for large calculations

         --------------------------------------------------------------------------------
19:53:42 Calculating damages...
         Number of Performance Groups in Asset Model: 30
         Number of Component Blocks: 30
         1 batches of Performance Groups prepared for damage assessment
19:53:42 Calculating damage for PG batch 1 with 30 blocks
19:53:42 Generating capacity variables ...
         2x56 random variables created.
19:53:42 Sampling capacities...
19:53:42 Raw samples are available
19:53:42 Successfully generated 1000 realizations.
19:53:42 Collecting required demand information...
19:53:42 Assembling demand data for calculation...
         
19:53:42 Evaluating damage states...
19:53:42 Calculating damage quantities...
         Raw damage calculation successful.
19:53:42 Applying damage processes...
19:53:42 Applying task...
         Damage process task successfully applied.
19:53:42 Applying task...
         Damage process task successfully applied.
         Damage process

In [52]:
# PAL.damage.save_sample(os.path.join(datDir, 'Damage_sample.csv'))

In [53]:
damage_sample = PAL.damage.save_sample()

print("Size of damage results: ", sys.getsizeof(damage_sample)/1024/1024, "MB")

         --------------------------------------------------------------------------------
19:53:42 Saving damage sample...
Size of damage results:  0.3090019226074219 MB


In [54]:
damage_sample.shape

(1000, 36)

#### Damage summary statistics of a component type

In [55]:
# component = 'B.20.22.031'
# damage_sample.describe([0.1, 0.5, 0.9]).T.loc[component,:].head(30)

In [56]:
damage_sample.head()

cmp B.10.71.002        B.10.71.041                  B.20.11.111                  C.10.11.001a             C.10.11.011a                       D.20.21.013a              D.20.21.013b              D.20.21.023a              D.20.21.023b              D.20.31.013b              D.30.31.013a D.30.41.011c  \
loc           1                  1                            1                             1                        1                                  1                         1                         1                         1                         1                         1            1   
dir           1      2           1           2                1           2                 1           2            1                2                 1            2            1            2            1            2            1            2            1            2            1            1   
ds            0      0           0    1      0    1           0    1      0    1            0           0            0    1           0    1            0            0            0            0            0            0            0            0            0            0            0            0   
0         100.0  100.0       100.0  0.0  100.0  0.0       100.0  0.0  100.0  0.0    76.155215   68.578668    81.316818  0.0  106.121952  0.0   930.380760  1546.143682  1121.286518  2283.051223  1560.108743  1335.510668  1219.922184  1458.993603  1145.575174   773.877160          1.0   926.303885   
1         100.0  100.0       100.0  0.0  100.0  0.0       100.0  0.0  100.0  0.0    74.987867   68.281866    46.091908  0.0  147.632513  0.0  1107.894562  1295.955637   998.017112  1413.769156   890.059594  1594.237155  1054.831453  2695.262829   850.814768   842.113284          1.0   992.825067   
2         100.0  100.0       100.0  0.0  100.0  0.0       100.0  0.0  100.0  0.0    91.191477  149.402919   105.103062  0.0  104.981595  0.0   830.903718   666.693700   811.912052   945.357245   760.143049   973.265521  1072.444395  1108.574843   678.161319   688.349330          1.0   669.181772   
3         100.0  100.0       100.0  0.0  100.0  0.0       100.0  0.0  100.0  0.0    60.372528  129.430671   129.585370  0.0   90.781653  0.0  1325.487642   932.342248   840.385738  1227.796297   878.122538   643.680249   737.285066   681.306833   834.547837   787.469046          1.0   767.843417   
4         100.0  100.0       100.0  0.0  100.0  0.0       100.0  0.0  100.0  0.0    76.983698   53.935893   166.310077  0.0  136.230554  0.0  1388.751176  1190.286631   958.895620  1591.539621   961.397256  1824.808106   698.261543  1182.862589   495.689676  1721.493755          1.0  1207.313990   

cmp              D.30.41.032c D.30.41.102b D.30.41.103a collapse excessiveRID      irreparable  
loc                         2            2            2        0            1                0  
dir            2            1            1            1        1            1    2           1  
ds             0            0            0            0        0            0    0           0  
0     661.907045         10.0          1.0          1.0      1.0          1.0  1.0         1.0  
1    1418.662927         10.0          1.0          1.0      1.0          1.0  1.0         1.0  
2    1638.807995         10.0          1.0          1.0      1.0          1.0  1.0         1.0  
3    1339.582581         10.0          1.0          1.0      1.0          1.0  1.0         1.0  
4    1077.407190         10.0          1.0          1.0      1.0          1.0  1.0         1.0

In [57]:
# damage_sample.to_csv('damage_sample.csv')

In [58]:
# damage_sample.columns

In [59]:
# component = 'D.10.14.011'
# # damage_sample.describe([0.1, 0.5, 0.9]).T.loc[component,:].head(30)
# dmg_plot = damage_sample.loc[:, component].groupby(level=[0,2], axis=1).sum().T

# px.bar(x=dmg_plot.index.get_level_values(1), y=dmg_plot.mean(axis=1), 
#        color=dmg_plot.index.get_level_values(0),
#        barmode='group',
#        labels={
#            'x':'Damage State',
#            'y':'Component Quantity [ft2]',
#            'color': 'Floor'
#        },
#        title=f'Mean Quantities of component {component} in each Damage State',
#        height=500
#       )

In [60]:
print('Probability of collapse: ', damage_sample[('collapse','0','1','0')].mean())
print()
print('Probability of irreparable damage: ', damage_sample[('irreparable','0','1','0')].mean())

Probability of collapse:  1.0

Probability of irreparable damage:  1.0


### Consequence mapping to damages

In [61]:
# let us prepare the map based on the component list

# we need to prepend 'DMG-' to the component names to tell pelicun to look for the damage of these components
drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
drivers = drivers[:-3]+drivers[-2:]

# we are looking at repair consequences in this example
# the components in P58 have consequence models under the same name
loss_models = cmp_marginals.index.unique().tolist()[:-3]

# We will define the replacement consequence in the following cell.
loss_models+=['replacement',]*2

# Assemble the DataFrame with the mapping information
# The column name identifies the type of the consequence model.
loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

loss_map

,BldgRepair
DMG-B.10.71.002,B.10.71.002
DMG-B.10.71.041,B.10.71.041
DMG-B.20.11.111,B.20.11.111
DMG-C.10.11.011a,C.10.11.011a
DMG-C.10.11.001a,C.10.11.001a
DMG-D.20.21.013a,D.20.21.013a
DMG-D.20.21.013b,D.20.21.013b
DMG-D.20.21.023a,D.20.21.023a
DMG-D.20.21.023b,D.20.21.023b
DMG-D.20.31.013b,D.20.31.013b


In [62]:
# load the consequence models
P58_data = PAL.get_default_data('bldg_repair_DB_FEMA_P58_2nd')

# get the consequences used by this assessment
P58_data_for_this_assessment = P58_data.loc[loss_map['BldgRepair'].values[:-2],:]

print(P58_data_for_this_assessment['Incomplete'].sum(), ' components have incomplete consequence models assigned.')

display(P58_data_for_this_assessment.head(30))

0.0  components have incomplete consequence models assigned.


DS1                                                 DS10                                DS11                                DS12                                DS13                                DS14                                DS15                               \
                      Family LongLeadTime                Theta_0   Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1   
B.10.71.002  Cost  lognormal          NaN    2134.17,1313.33|3,8  0.188729    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0    2.06801,1.27132|3,8  0.313239    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
B.10.71.041  Cost     normal          NaN         4340,1302|1,10  0.458612    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time     normal          0.0   3.38132,1.01347|1,10  0.522326    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
B.20.11.111  Cost  lognormal          NaN          2990,1840|2,6  0.256634    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
...                      ...          ...                    ...       ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...   
D.30.41.032c Time     normal          0.0    3.88194,3.17606|1,5  0.324289    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
D.30.41.102b Cost  lognormal          NaN          2915,2385|1,5  0.139425    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          1.0    3.42936,2.80584|1,5  0.286250    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
D.30.41.103a Cost  lognormal          NaN            770,630|1,5  0.342516    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0  0.905853,0.741147|1,5  0.424049    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   

                         DS2                                                     DS3                                                 DS4                                 DS5                              

### adding custom consequence functions

we need to define the replacement consequence for the collapse and irreparable damage cases.

In [63]:
# initialize the dataframe
additional_consequences = pd.DataFrame(
    columns = pd.MultiIndex.from_tuples([('Incomplete',''), 
                                         ('Quantity','Unit'), 
                                         ('DV', 'Unit'), 
                                         ('DS1', 'Theta_0')]),
    index=pd.MultiIndex.from_tuples([('replacement','Cost'), 
                                     ('replacement','Time')])
)

# add the data about replacement cost and time
additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]  

additional_consequences

Incomplete Quantity          DV       DS1
                                Unit        Unit   Theta_0
replacement Cost          0     1 EA    USD_2011  21600000
            Time          0     1 EA  worker_day     12500

### 3.4.3 Load component consequence data

In [64]:
# Load the loss model to pelicun
PAL.bldg_repair.load_model(
    [additional_consequences,
     "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
    loss_map)

         --------------------------------------------------------------------------------
19:53:42 Loading loss map for BldgRepair...
         Loss map successfully parsed.
         --------------------------------------------------------------------------------
19:53:42 Loading loss parameters for BldgRepair...
         Loss parameters successfully parsed.


### 3.4.4 Loss calculation

Loss calculation in Pelicun requires 
- a pre-assigned set of component consequence functions;
- a pre-assigned sample of demands and/or damages;
- and a loss mapping matrix

The sample size for the loss calculation is automatically inferred from the demand/damage sample size.

In [65]:
PAL.bldg_repair.calculate()

         --------------------------------------------------------------------------------
19:53:42 Calculating losses...
         
         Aggregating damage quantities...
         Successfully aggregated damage quantities.
         
         Calculating the median repair consequences...
         Successfully determined median repair consequences.
19:53:42 
         Considering deviations from the median values to obtain random DV sample...
         Preparing random variables for repair cost and time...
         
         12 random variables created.
         
         Successfully generated 1000 realizations of deviation from the median consequences.
         Successfully obtained DV sample.
19:53:43 Loss calculation successful.


In [66]:
# PAL.bldg_repair.save_sample(os.path.join(datDir, 'loss_sample.csv'))

In [67]:
loss_sample = PAL.bldg_repair.sample
loss_sample_cost = loss_sample['COST']
loss_sample_time = loss_sample['TIME']
loss_sample_cost.head()

loss B.10.71.041      B.20.11.111      C.10.11.011a     
dmg  B.10.71.041      B.20.11.111      C.10.11.011a     
ds             1                1                 1     
loc            1                1                 1     
dir            1    2           1    2            1    2
0            0.0  0.0         0.0  0.0          0.0  0.0
1            0.0  0.0         0.0  0.0          0.0  0.0
2            0.0  0.0         0.0  0.0          0.0  0.0
3            0.0  0.0         0.0  0.0          0.0  0.0
4            0.0  0.0         0.0  0.0          0.0  0.0

In [68]:
# loss_sample_cost.to_csv('loss_sample_cost.csv')
# loss_sample_time.to_csv('loss_sample_time.csv')

**Aggregate losses**

Aggregating losses for repair costs is straightforward, but repair times are less trivial. Pelicun adopts the method from FEMA P-58 and provides two bounding values for aggregate repair times:
- **parallel** assumes that repairs are conducted in parallel across locations. In each location, repairs are assumed to be sequential. This translates to aggregating component repair times by location and choosing the longest resulting aggregate value across locations.
- **sequential** assumes repairs are performed sequentially across locations and within each location. This translates to aggregating component repair times across the entire building.

The parallel option is considered a lower bound and the sequential is an upper bound of the real repair time. Pelicun automatically calculates both options for all (i.e., not only FEMA P-58) analyses.

In [69]:
agg_DF = PAL.bldg_repair.aggregate_losses()
agg_DF.head()

         --------------------------------------------------------------------------------
19:53:43 Aggregating repair consequences...
19:53:43 Repair consequences successfully aggregated.


repair_cost repair_time           
                 parallel sequential
0         0.0         0.0        0.0
1         0.0         0.0        0.0
2         0.0         0.0        0.0
3         0.0         0.0        0.0
4         0.0         0.0        0.0

In [70]:
# agg_DF.to_csv('aggregate_loss.csv')

In [71]:
agg_DF.describe()

repair_cost  repair_time             
                       parallel   sequential
count  1000.000000  1000.000000  1000.000000
mean    249.132653     0.212821     0.212821
std    1057.524422     0.957683     0.957683
min       0.000000     0.000000     0.000000
25%       0.000000     0.000000     0.000000
50%       0.000000     0.000000     0.000000
75%       0.000000     0.000000     0.000000
max    9896.996315    12.128750    12.128750

# 4 Concise version

All scripts required to run this example are collected to a few cells below for your convenience.

In [72]:
#parameters
sample_size = 10000
delta_y = 0.0075
stripe = '1'
PAL = Assessment({"PrintLog": True, "Seed": 415,})

pelicun 3.1 | 

System Information:
         local time zone: +0545
         start time: 2023-03-20T19:53:43
         python: 3.8.2 (default, Mar 25 2020, 11:22:43) 
         [Clang 4.0.1 (tags/RELEASE_401/final)]
         numpy: 1.21.4
         pandas: 1.4.4
         
         --------------------------------------------------------------------------------
19:53:43 Assessement Started


## 4.1 Pre-processing

In [73]:
# prepare demand input
raw_demands = convert_to_MultiIndex(df_comb, axis=0)
raw_demands.index.names = ['stripe','type','loc','dir']

# prepare the demand input for pelicun
stripe_demands = raw_demands.loc[stripe,:]

# units - - - - - - - - - - - - - - - - - - - - - - - -  
stripe_demands.insert(0, 'Units',"")
stripe_demands.loc['PFA','Units'] = 'g'
stripe_demands.loc['PID','Units'] = 'rad'

# distribution family  - - - - - - - - - - - - - - - - -  
stripe_demands.insert(1, 'Family',"")
stripe_demands['Family'] = 'lognormal'

# distribution parameters  - - - - - - - - - - - - - - -
stripe_demands.rename(columns = {'median': 'Theta_0'}, inplace=True)
stripe_demands.rename(columns = {'log_std': 'Theta_1'}, inplace=True)


# prepare a correlation matrix that represents perfect correlation
ndims = stripe_demands.shape[0]
demand_types = stripe_demands.index 

perfect_CORR = pd.DataFrame(
    np.ones((ndims, ndims)),
    columns = demand_types,
    index = demand_types)

# prepare additional fragility and consequence data ahead of time
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
# baselineID = 's4_96x48'
cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                    baselineID,
                                                    'ComponentsList',
                                                    'components_list_marginals.csv']), index_col=1)

# add missing data to P58 damage model
P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')
cmp_list = cmp_marginals.index.unique().values[:-3]

# now take those components that are incomplete, and add the missing information
additional_fragility_db = P58_data.loc[cmp_list,:].loc[P58_data.loc[cmp_list,'Incomplete'] == 1].sort_index()

# # D2022.013a, 023a, 023b - Heating, hot water piping and bracing
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc[['D.20.22.013a','D.20.22.023a','D.20.22.023b'],
#                             [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.5

# # D2031.013b - Sanitary Waste piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.31.013b',('LS1','Theta_1')] = 0.5

# # D2061.013b - Steam piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.61.013b',('LS1','Theta_1')] = 0.5

# # D3031.013i - Chiller
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_1')] = 0.5

# # D3031.023i - Cooling Tower
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_1')] = 0.5

# # D3052.013i - Air Handling Unit
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_1')] = 0.5

# prepare the extra damage models for collapse and irreparable damage
additional_fragility_db.loc[
    'excessiveRID', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

additional_fragility_db.loc[
    'excessiveRID', [('LS1','Family'),
                    ('LS1','Theta_0'),
                    ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

additional_fragility_db.loc[
    'irreparable', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

additional_fragility_db.loc[
    'irreparable', ('LS1','Theta_0')] = 1e10

additional_fragility_db.loc[
    'collapse', [('Demand','Directional'),
                 ('Demand','Offset'),
                 ('Demand','Type'), 
                 ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

additional_fragility_db.loc[
    'collapse', [('LS1','Family'),
                 ('LS1','Theta_0'),
                 ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

# Now we can set the incomplete flag to 0 for these components
additional_fragility_db['Incomplete'] = 0

# create the additional consequence models
additional_consequences = pd.DataFrame(
    columns = pd.MultiIndex.from_tuples([
        ('Incomplete',''), ('Quantity','Unit'), ('DV', 'Unit'), ('DS1', 'Theta_0')]),
    index=pd.MultiIndex.from_tuples([
        ('replacement','Cost'), ('replacement','Time')])
)

additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]

## 4.2 Demands

In [74]:
# initialize a pelicun Assessment
# PAL = Assessment({"PrintLog": True, "Seed": 415,})

# load the demand model
PAL.demand.load_model({'marginals': stripe_demands,
                       'correlation': perfect_CORR})

# generate samples
PAL.demand.generate_sample({"SampleSize": sample_size})

# add residual drift and Sa
demand_sample = PAL.demand.save_sample()

RID = PAL.demand.estimate_RID(demand_sample['PID'], {'yield_drift': delta_y})
demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[int(stripe)-1]

# add units to the data 
demand_sample_ext.T.insert(0, 'Units',"")

# PFA and SA are in "g" in this example, while PID and RID are "rad"
demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

PAL.demand.load_sample(demand_sample_ext)

         --------------------------------------------------------------------------------
19:53:43 Loading demand model...
         Demand model successfully loaded.
         --------------------------------------------------------------------------------
19:53:43 Generating sample from demand variables...
         
         6 random variables created.
         
         Correlations between 6 random variables successfully defined.
         
         Successfully generated 10000 realizations.
         --------------------------------------------------------------------------------
         --------------------------------------------------------------------------------
19:53:43 Loading demand data...
WARNING in pandas/core/indexing.py at line 2120
In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)

         Demand data successfully parsed.
         Demand units successfully parsed.


## 4.3 Damage

In [75]:
# specify number of stories
PAL.stories = 1

# load component definitions
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
PAL.asset.load_cmp_model({'marginals': cmp_marginals})

# generate sample
PAL.asset.generate_cmp_sample(sample_size)

# load the models into pelicun
PAL.damage.load_damage_model([
    additional_fragility_db,  # This is the extra fragility data we've just created
    'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
])

# prescribe the damage process
dmg_process = {
    "1_collapse": {
        "DS1": "ALL_NA"
    },
    "2_excessiveRID": {
        "DS1": "irreparable_DS1"
    }
}

# calculate damages
PAL.damage.calculate(dmg_process=dmg_process)

         --------------------------------------------------------------------------------
19:53:43 Loading component model...
         
         Parsing model file to characterize each component block
         Model parameters successfully parsed. 30 performance groups identified
         Converting model parameters to internal units...
         Model parameters successfully loaded.
         
         Component model marginal distributions:
                              Units Family    Theta_0  Theta_1  Blocks
         cmp          loc dir                                         
         B.10.71.002  1   1      SF    NaN   9.290304      NaN     1.0
                          2      SF    NaN   9.290304      NaN     1.0
         B.10.71.041  1   1      LF    NaN  30.480000      NaN     1.0
                          2      LF    NaN  30.480000      NaN     1.0
         B.20.11.111  1   1      SF    NaN   9.290304      NaN     1.0
         ...                    ...    ...        ...     

## 4.4 Losses

In [76]:
# create the loss map
drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
drivers = drivers[:-3]+drivers[-2:]

loss_models = cmp_marginals.index.unique().tolist()[:-3] +['replacement',]*2

loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

# load the loss model
PAL.bldg_repair.load_model(
    [additional_consequences,
     "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
    loss_map)

# perform the calculation
PAL.bldg_repair.calculate()

# get the aggregate losses
agg_DF = PAL.bldg_repair.aggregate_losses()

agg_DF.describe([0.1, 0.5, 0.9])

         --------------------------------------------------------------------------------
19:53:43 Loading loss map for BldgRepair...
         Loss map successfully parsed.
         --------------------------------------------------------------------------------
19:53:43 Loading loss parameters for BldgRepair...
         Loss parameters successfully parsed.
         --------------------------------------------------------------------------------
19:53:44 Calculating losses...
         
         Aggregating damage quantities...
         Successfully aggregated damage quantities.
         
         Calculating the median repair consequences...
         Successfully determined median repair consequences.
19:53:44 
         Considering deviations from the median values to obtain random DV sample...
         Preparing random variables for repair cost and time...
         
         14 random variables created.
         
         Successfully generated 10000 realizations of deviation from the

repair_cost   repair_time              
                         parallel    sequential
count  10000.000000  10000.000000  10000.000000
mean     261.001210      0.208593      0.208872
std     1153.141629      0.941952      0.943559
min        0.000000      0.000000      0.000000
10%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
90%        0.000000      0.000000      0.000000
max    21369.857565     16.546090     16.546090

In [77]:
agg_DF

repair_cost repair_time           
                    parallel sequential
0            0.0         0.0        0.0
1            0.0         0.0        0.0
2            0.0         0.0        0.0
3            0.0         0.0        0.0
4            0.0         0.0        0.0
...          ...         ...        ...
9995         0.0         0.0        0.0
9996         0.0         0.0        0.0
9997         0.0         0.0        0.0
9998         0.0         0.0        0.0
9999         0.0         0.0        0.0

[10000 rows x 3 columns]

In [78]:
from Run_pelicun_v3p1 import compile_demand_data

df_raw = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])

In [79]:
df_raw.head()

Family   Theta_0   Theta_1
stripe type loc dir                               
1      PID  1   1    lognormal  0.000408  0.000201
                2    lognormal  0.000602  0.000246
       PFA  0   1    lognormal  0.066539  0.026599
                2    lognormal  0.067214  0.033207
            1   1    lognormal  0.095921  0.053826

In [80]:
#parameters
sample_size = 1000
delta_y = 0.0075
stripe = 3
PAL = Assessment({"PrintLog": False, "Seed": 415,})
raw_demands = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])
baselineID = 's4_96x48'

In [81]:
def loss_assessment(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000):
    # prepare the demand input for pelicun
    stripe_demands = raw_demands.loc[str(stripe),:]

    # units - - - - - - - - - - - - - - - - - - - - - - - -  
    stripe_demands.insert(0, 'Units',"")
    stripe_demands.loc['PFA','Units'] = 'g'
    stripe_demands.loc['PID','Units'] = 'rad'


    # prepare a correlation matrix that represents perfect correlation
    ndims = stripe_demands.shape[0]
    demand_types = stripe_demands.index 

    perfect_CORR = pd.DataFrame(
        np.ones((ndims, ndims)),
        columns = demand_types,
        index = demand_types)

    # prepare additional fragility and consequence data ahead of time
    # cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)

    cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                        baselineID,
                                                        'ComponentsList',
                                                        'components_list_marginals.csv']), index_col=1)

    # add missing data to P58 damage model
    P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')
    cmp_list = cmp_marginals.index.unique().values[:-3]

    # now take those components that are incomplete, and add the missing information
    additional_fragility_db = P58_data.loc[cmp_list,:].loc[P58_data.loc[cmp_list,'Incomplete'] == 1].sort_index()

    # prepare the extra damage models for collapse and irreparable damage
    additional_fragility_db.loc[
        'excessiveRID', [('Demand','Directional'),
                        ('Demand','Offset'),
                        ('Demand','Type'), 
                        ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

    additional_fragility_db.loc[
        'excessiveRID', [('LS1','Family'),
                        ('LS1','Theta_0'),
                        ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

    additional_fragility_db.loc[
        'irreparable', [('Demand','Directional'),
                        ('Demand','Offset'),
                        ('Demand','Type'), 
                        ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

    additional_fragility_db.loc[
        'irreparable', ('LS1','Theta_0')] = 1e10

    additional_fragility_db.loc[
        'collapse', [('Demand','Directional'),
                     ('Demand','Offset'),
                     ('Demand','Type'), 
                     ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

    additional_fragility_db.loc[
        'collapse', [('LS1','Family'),
                     ('LS1','Theta_0'),
                     ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

    # Now we can set the incomplete flag to 0 for these components
    additional_fragility_db['Incomplete'] = 0

    # create the additional consequence models
    additional_consequences = pd.DataFrame(
        columns = pd.MultiIndex.from_tuples([
            ('Incomplete',''), ('Quantity','Unit'), ('DV', 'Unit'), ('DS1', 'Theta_0')]),
        index=pd.MultiIndex.from_tuples([
            ('replacement','Cost'), ('replacement','Time')])
    )

    additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
    additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]

    # load the demand model
    PAL.demand.load_model({'marginals': stripe_demands,
                           'correlation': perfect_CORR})

    # generate samples
    PAL.demand.generate_sample({"SampleSize": sample_size})

    # add residual drift and Sa
    demand_sample = PAL.demand.save_sample()

    RID = PAL.demand.estimate_RID(demand_sample['PID'], {'yield_drift': delta_y})
    demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

    Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
    demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[stripe-1]

    # add units to the data 
    demand_sample_ext.T.insert(0, 'Units',"")

    # PFA and SA are in "g" in this example, while PID and RID are "rad"
    demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
    demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

    PAL.demand.load_sample(demand_sample_ext)

    # specify number of stories
    PAL.stories = 4

    # load component definitions
    # cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
    PAL.asset.load_cmp_model({'marginals': cmp_marginals})

    # generate sample
    PAL.asset.generate_cmp_sample(sample_size)

    # load the models into pelicun
    PAL.damage.load_damage_model([
        additional_fragility_db,  # This is the extra fragility data we've just created
        'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
    ])

    # prescribe the damage process
    dmg_process = {
        "1_collapse": {
            "DS1": "ALL_NA"
        },
        "2_excessiveRID": {
            "DS1": "irreparable_DS1"
        }
    }

    # calculate damages
    PAL.damage.calculate(dmg_process=dmg_process)

    # create the loss map
    drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
    drivers = drivers[:-3]+drivers[-2:]

    loss_models = cmp_marginals.index.unique().tolist()[:-3] +['replacement',]*2

    loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

    # load the loss model
    PAL.bldg_repair.load_model(
        [additional_consequences,
         "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
        loss_map)

    # perform the calculation
    PAL.bldg_repair.calculate()

    # get the aggregate losses
    agg_DF = PAL.bldg_repair.aggregate_losses()
    return agg_DF

In [82]:
a = loss_assessment(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)
a

KeyError: '3'

In [ ]:
from Run_pelicun_v3p1 import pelicun_assessment1

b = pelicun_assessment1(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)

In [ ]:
b